In [1]:
import requests 
import pandas as pd

c:\Users\alver\anaconda3\lib\site-packages\pandas\core\computation\expressions.py:21: UserWarning: Pandas requires version '2.8.4' or newer of 'numexpr' (version '2.8.1' currently installed).
  from pandas.core.computation.check import NUMEXPR_INSTALLED
c:\Users\alver\anaconda3\lib\site-packages\pandas\core\arrays\masked.py:60: UserWarning: Pandas requires version '1.3.6' or newer of 'bottleneck' (version '1.3.4' currently installed).
  from pandas.core import (


In [22]:
API_KEY = 'OGA9PANJZTKOUAW1'

finance_symbols = ['JPM', 'BAC', 'C', 'WFC', 'GS']
industrial_symbols = ['MMM', 'ITW', 'EMR', 'PH', 'TEX']
consumer_symbols = ['WMT', 'EBAY', 'JD', 'BABA', 'ETSY']
tech_symbols = ['ORCL', 'SAP', 'ADBE', 'TEAM', 'MSFT']

our_stocks = ['CRM', 'AMZN', 'JPM', 'GS', 'CAT']
test = ['CRM']

In [12]:
def get_eps(symbol):
    url = f'https://www.alphavantage.co/query'
    params = {
        'function': 'EARNINGS',
        'symbol': symbol,
        'apikey': API_KEY
    }
    response = requests.get(url, params=params)
    data = response.json()
    print(data)
    
    eps = []
    
    # Extract EPS data for 2016
    if 'quarterlyEarnings' in data:
        for earnings in data['annualEarnings']:
            fiscal_date = earnings['fiscalDateEnding']
            if fiscal_date.startswith('2016'):
                eps_value = earnings.get('reportedEPS', None)  # Check for reportedEPS
                if eps_value is not None:
                    eps.append(float(eps_value))

    return eps

def industry_eps(industry_name):
    all_eps = []

    for symbol in industry_name:
        eps_values = get_eps(symbol)
        all_eps.extend(eps_values)

    # Calculate average EPS
    if all_eps:
        average_eps = sum(all_eps) / len(all_eps)
        print(industry_name)
        print(all_eps)
        print(f'Average EPS for finance sector in 2016: {average_eps:.2f}')
    else:
        print('No EPS data found for the specified companies in 2016.')

In [13]:
industry_eps(finance_symbols)

{'symbol': 'JPM', 'annualEarnings': [{'fiscalDateEnding': '2024-09-30', 'reportedEPS': '13.07'}, {'fiscalDateEnding': '2023-12-31', 'reportedEPS': '16.22'}, {'fiscalDateEnding': '2022-12-31', 'reportedEPS': '12.44'}, {'fiscalDateEnding': '2021-12-31', 'reportedEPS': '15.44'}, {'fiscalDateEnding': '2020-12-31', 'reportedEPS': '8.87'}, {'fiscalDateEnding': '2019-12-31', 'reportedEPS': '10.41'}, {'fiscalDateEnding': '2018-12-31', 'reportedEPS': '8.87'}, {'fiscalDateEnding': '2017-12-31', 'reportedEPS': '6.88'}, {'fiscalDateEnding': '2016-12-31', 'reportedEPS': '6.06'}, {'fiscalDateEnding': '2015-12-31', 'reportedEPS': '5.99'}, {'fiscalDateEnding': '2014-12-31', 'reportedEPS': '5.29'}, {'fiscalDateEnding': '2013-12-31', 'reportedEPS': '6.01'}, {'fiscalDateEnding': '2012-12-31', 'reportedEPS': '5.27'}, {'fiscalDateEnding': '2011-12-31', 'reportedEPS': '4.47'}, {'fiscalDateEnding': '2010-12-31', 'reportedEPS': '3.6'}, {'fiscalDateEnding': '2009-12-31', 'reportedEPS': '2.24'}, {'fiscalDateEnd

In [23]:
industry_eps(industrial_symbols)

{'symbol': 'MMM', 'annualEarnings': [{'fiscalDateEnding': '2024-09-30', 'reportedEPS': '6.3'}, {'fiscalDateEnding': '2023-12-31', 'reportedEPS': '9.24'}, {'fiscalDateEnding': '2022-12-31', 'reportedEPS': '10.1'}, {'fiscalDateEnding': '2021-12-31', 'reportedEPS': '10.12'}, {'fiscalDateEnding': '2020-12-31', 'reportedEPS': '8.75'}, {'fiscalDateEnding': '2019-12-31', 'reportedEPS': '9.16'}, {'fiscalDateEnding': '2018-12-31', 'reportedEPS': '9.98'}, {'fiscalDateEnding': '2017-12-31', 'reportedEPS': '8.84'}, {'fiscalDateEnding': '2016-12-31', 'reportedEPS': '8.16'}, {'fiscalDateEnding': '2015-12-31', 'reportedEPS': '7.68'}, {'fiscalDateEnding': '2014-12-31', 'reportedEPS': '7.49'}, {'fiscalDateEnding': '2013-12-31', 'reportedEPS': '6.72'}, {'fiscalDateEnding': '2012-12-31', 'reportedEPS': '6.35'}, {'fiscalDateEnding': '2011-12-31', 'reportedEPS': '5.96'}, {'fiscalDateEnding': '2010-12-31', 'reportedEPS': '5.75'}, {'fiscalDateEnding': '2009-12-31', 'reportedEPS': '4.68'}, {'fiscalDateEnding'

In [24]:
industry_eps(consumer_symbols)

{'symbol': 'WMT', 'annualEarnings': [{'fiscalDateEnding': '2024-10-31', 'reportedEPS': '1.27'}, {'fiscalDateEnding': '2024-01-31', 'reportedEPS': '5.44'}, {'fiscalDateEnding': '2023-01-31', 'reportedEPS': '6.28'}, {'fiscalDateEnding': '2022-01-31', 'reportedEPS': '6.45'}, {'fiscalDateEnding': '2021-01-31', 'reportedEPS': '5.54'}, {'fiscalDateEnding': '2020-01-31', 'reportedEPS': '4.94'}, {'fiscalDateEnding': '2019-01-31', 'reportedEPS': '4.92'}, {'fiscalDateEnding': '2018-01-31', 'reportedEPS': '4.41'}, {'fiscalDateEnding': '2017-01-31', 'reportedEPS': '4.33'}, {'fiscalDateEnding': '2016-01-31', 'reportedEPS': '4.59'}, {'fiscalDateEnding': '2015-01-31', 'reportedEPS': '5.07'}, {'fiscalDateEnding': '2014-01-31', 'reportedEPS': '5.13'}, {'fiscalDateEnding': '2013-01-31', 'reportedEPS': '5.02'}, {'fiscalDateEnding': '2012-01-31', 'reportedEPS': '4.48'}, {'fiscalDateEnding': '2011-01-31', 'reportedEPS': '4.09'}, {'fiscalDateEnding': '2010-01-31', 'reportedEPS': '3.66'}, {'fiscalDateEnding'

In [19]:
industry_eps(tech_symbols)

{'symbol': 'ORCL', 'annualEarnings': [{'fiscalDateEnding': '2024-08-31', 'reportedEPS': '1.39'}, {'fiscalDateEnding': '2024-05-31', 'reportedEPS': '5.57'}, {'fiscalDateEnding': '2023-05-31', 'reportedEPS': '5.13'}, {'fiscalDateEnding': '2022-05-31', 'reportedEPS': '4.91'}, {'fiscalDateEnding': '2021-05-31', 'reportedEPS': '4.69'}, {'fiscalDateEnding': '2020-05-31', 'reportedEPS': '3.88'}, {'fiscalDateEnding': '2019-05-31', 'reportedEPS': '3.54'}, {'fiscalDateEnding': '2018-05-31', 'reportedEPS': '3.14'}, {'fiscalDateEnding': '2017-05-31', 'reportedEPS': '2.74'}, {'fiscalDateEnding': '2016-05-31', 'reportedEPS': '2.61'}, {'fiscalDateEnding': '2015-05-31', 'reportedEPS': '2.77'}, {'fiscalDateEnding': '2014-05-31', 'reportedEPS': '2.88'}, {'fiscalDateEnding': '2013-05-31', 'reportedEPS': '2.69'}, {'fiscalDateEnding': '2012-05-31', 'reportedEPS': '2.46'}, {'fiscalDateEnding': '2011-05-31', 'reportedEPS': '2.22'}, {'fiscalDateEnding': '2010-05-31', 'reportedEPS': '1.67'}, {'fiscalDateEnding

In [ ]:
# def get_earnings_growth(symbol):
#     API_KEY = 'YOUR_API_KEY'  # Replace with your actual API key
#     url = f'https://www.alphavantage.co/query'
#     params = {
#         'function': 'EARNINGS',
#         'symbol': symbol,
#         'apikey': API_KEY
#     }
    
#     response = requests.get(url, params=params)
#     data = response.json()

#     # Extract earnings data
#     earnings = {}
#     if 'annualEarnings' in data:
#         for entry in data['annualEarnings']:
#             year = entry['fiscalDateEnding'][:4]
#             eps = float(entry['reportedEPS'])
#             earnings[year] = eps

#     return earnings

# for stock in our_stocks:
#     print(stock)
#     print(get_earnings_growth(stock))
#     print('')

CRM
{'2024': 8.21, '2023': 5.25, '2022': 4.8, '2021': 4.92, '2020': 3.0, '2019': 2.58, '2018': 1.35, '2017': 1.0, '2016': 0.75, '2015': 0.52, '2014': 0.35, '2013': 0.41, '2012': 0.35, '2011': 0.31, '2010': 0.16, '2009': 0.09, '2008': 0.05, '2007': 0.07, '2006': 0.04, '2005': 0.0173}

AMZN
{'2024': 3.67, '2023': 2.9, '2022': 0.97, '2021': 2.15, '2020': 2.09, '2019': 1.14, '2018': 1.0, '2017': 0.23, '2016': 0.25, '2015': 0.06, '2014': -0.03, '2013': -0.07, '2012': 0.02, '2011': 0.07, '2010': 0.13, '2009': 0.1, '2008': 0.08, '2007': 0.05, '2006': 0.12, '2005': 0.11, '2004': 0.05, '2003': 0.04, '2002': -0.1024, '2001': 0.06, '2000': -0.06, '1999': -0.06, '1998': -0.25, '1997': -0.08}

JPM
{'2024': 13.07, '2023': 16.22, '2022': 12.44, '2021': 15.44, '2020': 8.87, '2019': 10.41, '2018': 8.87, '2017': 6.88, '2016': 6.06, '2015': 5.99, '2014': 5.29, '2013': 6.01, '2012': 5.27, '2011': 4.47, '2010': 3.6, '2009': 2.24, '2008': 1.4, '2007': 4.37, '2006': 3.86, '2005': 2.95, '2004': 3.01, '2003': 

In [3]:
def get_earnings_growth(symbol):
    API_KEY = 'YOUR_API_KEY'  # Replace with your actual API key
    url = f'https://www.alphavantage.co/query'
    params = {
        'function': 'EARNINGS',
        'symbol': symbol,
        'apikey': API_KEY
    }
    
    response = requests.get(url, params=params)
    data = response.json()

    # Extract earnings data
    earnings = {}
    if 'annualEarnings' in data:
        for entry in data['annualEarnings']:
            year = entry['fiscalDateEnding'][:4]
            eps = float(entry['reportedEPS'])
            if year in ['2010', '2011', '2012', '2013', '2014', '2015', '2016']:
                earnings[year] = eps

    return earnings

for stock in our_stocks:
    print(stock)
    print(get_earnings_growth(stock))
    print('')

CRM
{'2016': 0.75, '2015': 0.52, '2014': 0.35, '2013': 0.41, '2012': 0.35, '2011': 0.31, '2010': 0.16}

AMZN
{'2016': 0.25, '2015': 0.06, '2014': -0.03, '2013': -0.07, '2012': 0.02, '2011': 0.07, '2010': 0.13}

JPM
{'2016': 6.06, '2015': 5.99, '2014': 5.29, '2013': 6.01, '2012': 5.27, '2011': 4.47, '2010': 3.6}

GS
{'2016': 16.36, '2015': 18.27, '2014': 17.07, '2013': 15.47, '2012': 14.15, '2011': 4.41, '2010': 13.14}

CAT
{'2016': 3.44, '2015': 4.48, '2014': 6.37, '2013': 5.79, '2012': 8.63, '2011': 7.39, '2010': 4.04}



In [44]:
def get_financial_statements(symbol):
    url = f'https://www.alphavantage.co/query'
    params = {
        'function': 'BALANCE_SHEET',
        'symbol': symbol,
        'apikey': API_KEY
    }
    response = requests.get(url, params=params)
    data = response.json()
    print(data)
    return data.get('annualReports', [])

def get_debt_to_equity_2016(symbol):
    financials = get_financial_statements(symbol)
    for report in financials:
        if 'fiscalDateEnding' in report and '2016' in report['fiscalDateEnding']:
            print(report)
            total_debt = float(report.get('currentDebt', 0))
            total_shareholder_equity = float(report.get('shortTermDebt', 0)) + float(report.get('longTermDebt', 0))
            if total_shareholder_equity > 0:  # Avoid division by zero
                return total_debt / total_shareholder_equity
    return None  # If 2016 data is not found or cannot calculate

# Example use for multiple symbols
def industry_debt_to_equity(industry_symbols):
    all_ratios = []
    for symbol in industry_symbols:
        ratio = get_debt_to_equity_2016(symbol)
        if ratio is not None:
            all_ratios.append(ratio)

    if all_ratios:
        average_ratio = sum(all_ratios) / len(all_ratios)
        print(f'Average Debt-to-Equity Ratio for 2016: {average_ratio:.2f}')
    else:
        print('No 2016 Debt-to-Equity data found for the specified companies.')

In [40]:
industry_debt_to_equity(finance_symbols)

{'symbol': 'JPM', 'annualReports': [{'fiscalDateEnding': '2023-12-31', 'reportedCurrency': 'USD', 'totalAssets': '3875393000000', 'totalCurrentAssets': '628136000000', 'cashAndCashEquivalentsAtCarryingValue': '604551000000', 'cashAndShortTermInvestments': '624151000000', 'inventory': '-1856346000000', 'currentNetReceivables': 'None', 'totalNonCurrentAssets': '92528000000', 'propertyPlantEquipment': '10663000000', 'accumulatedDepreciationAmortizationPPE': 'None', 'intangibleAssets': '58834000000', 'intangibleAssetsExcludingGoodwill': '6200000000', 'goodwill': '52634000000', 'investments': '625148000000', 'longTermInvestments': '973946000000', 'shortTermInvestments': '19600000000', 'otherCurrentAssets': '824984000000', 'otherNonCurrentAssets': 'None', 'totalLiabilities': '3547515000000', 'totalCurrentLiabilities': '53244000000', 'currentAccountsPayable': 'None', 'deferredRevenue': 'None', 'currentDebt': '44712000000', 'shortTermDebt': '44712000000', 'totalNonCurrentLiabilities': '6800000

In [43]:
industry_debt_to_equity(industrial_symbols)

{'fiscalDateEnding': '2016-12-31', 'reportedCurrency': 'USD', 'totalAssets': '32906000000', 'totalCurrentAssets': '11726000000', 'cashAndCashEquivalentsAtCarryingValue': '2398000000', 'cashAndShortTermInvestments': '2678000000', 'inventory': '3385000000', 'currentNetReceivables': '4392000000', 'totalNonCurrentAssets': '21198000000', 'propertyPlantEquipment': '8516000000', 'accumulatedDepreciationAmortizationPPE': '14983000000', 'intangibleAssets': '11486000000', 'intangibleAssetsExcludingGoodwill': '2320000000', 'goodwill': '9166000000', 'investments': '574000000', 'longTermInvestments': '146000000', 'shortTermInvestments': '280000000', 'otherCurrentAssets': '450000000', 'otherNonCurrentAssets': '1178000000', 'totalLiabilities': '22563000000', 'totalCurrentLiabilities': '6219000000', 'currentAccountsPayable': '1798000000', 'deferredRevenue': '566000000', 'currentDebt': '972000000', 'shortTermDebt': '963000000', 'totalNonCurrentLiabilities': '16793000000', 'capitalLeaseObligations': '45

In [45]:
industry_debt_to_equity(consumer_symbols)

{'symbol': 'WMT', 'annualReports': [{'fiscalDateEnding': '2024-01-31', 'reportedCurrency': 'USD', 'totalAssets': '252399000000', 'totalCurrentAssets': '76877000000', 'cashAndCashEquivalentsAtCarryingValue': '9867000000', 'cashAndShortTermInvestments': '18316000000', 'inventory': '54892000000', 'currentNetReceivables': '8796000000', 'totalNonCurrentAssets': '179622000000', 'propertyPlantEquipment': '110810000000', 'accumulatedDepreciationAmortizationPPE': '109049000000', 'intangibleAssets': '32213000000', 'intangibleAssetsExcludingGoodwill': '4100000000', 'goodwill': '28113000000', 'investments': '1200000000', 'longTermInvestments': 'None', 'shortTermInvestments': '8449000000', 'otherCurrentAssets': '3322000000', 'otherNonCurrentAssets': '17071000000', 'totalLiabilities': '168538000000', 'totalCurrentLiabilities': '92415000000', 'currentAccountsPayable': '56812000000', 'deferredRevenue': 'None', 'currentDebt': '5050000000', 'shortTermDebt': '878000000', 'totalNonCurrentLiabilities': '54

ValueError: could not convert string to float: 'None'

In [46]:
industry_debt_to_equity(tech_symbols)

{'symbol': 'ORCL', 'annualReports': [{'fiscalDateEnding': '2024-05-31', 'reportedCurrency': 'USD', 'totalAssets': '140976000000', 'totalCurrentAssets': '22554000000', 'cashAndCashEquivalentsAtCarryingValue': '10454000000', 'cashAndShortTermInvestments': '10661000000', 'inventory': '334000000', 'currentNetReceivables': '7874000000', 'totalNonCurrentAssets': '118422000000', 'propertyPlantEquipment': '21536000000', 'accumulatedDepreciationAmortizationPPE': '13282000000', 'intangibleAssets': '69120000000', 'intangibleAssetsExcludingGoodwill': '6890000000', 'goodwill': '62230000000', 'investments': '2930000000', 'longTermInvestments': 'None', 'shortTermInvestments': '207000000', 'otherCurrentAssets': '2864000000', 'otherNonCurrentAssets': '15493000000', 'totalLiabilities': '132272000000', 'totalCurrentLiabilities': '31544000000', 'currentAccountsPayable': '2357000000', 'deferredRevenue': '21092000000', 'currentDebt': '10605000000', 'shortTermDebt': '11905000000', 'totalNonCurrentLiabilities